In [24]:
import random
import pickle
import os
import cv2
import numpy as np
import tensorflow
from PIL import Image
from matplotlib import pyplot as plt
from patchify import patchify
from enum import Enum
import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from simple_multi_unet_model import *

In [25]:
weights = [0.1666, 0.1666, 0.1666, 0.1666, 0.1666, 0.1666]
dice_loss = sm.losses.DiceLoss(class_weights=weights)
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


In [26]:
path_model = r"C:\Users\jiric\Documents\VUT\BP\DATASETS\CUSTOM_CZ_DATASET\MODELS\model_512_40_DSV30LITE_fd.h5"
save_path = r"C:\Users\jiric\Documents\VUT\BP\OBRAZKY\General\A star\PATH_C\UBRO93_pred2.png"
# image_path = r"C:\Users\jiric\Documents\VUT\BP\DATASETS\Custom_CZ_sample_work\Original Tiles\VYSKOV12\WRTO24.2022.VYSK12.jpg"
# save_path = r"C:\Users\jiric\Documents\VUT\BP\OBRAZKY\BIG_FORMAT_RESULTS\CUSTOM\TRAIN RESULTS\VY12_512_40_DSV30LITE_fd.png"
image_path = r"C:\Users\jiric\Documents\VUT\BP\OBRAZKY\General\A star\PATH_C\UBRO93_img2.jpg"
save_bool = 1

model = tensorflow.keras.models.load_model(path_model, custom_objects={'jacard_coef':jacard_coef, 'dice_loss_plus_1focal_loss':total_loss})

patch_size = 512

In [28]:
colormap = {
    0: [50, 50, 50],        # Unlabelled
    1: [50, 255, 167],      # Vegetation
    2: [255, 140, 0],     # Unpaved road
    3: [109, 50, 255],      # Paved area
    4: [255, 225, 50],      # Building
    5: [50, 167, 255],       # Water
    6: [166, 117, 86]         # Land
}

In [29]:
matrix = np.array((10, 10, 10))
print(matrix[0])
matrix_f = matrix.astype(np.float32)
print(matrix_f[0])

10
10.0


In [30]:
# image_path = "C:/Users/jiric/Documents/VUT/BP/DATASETS/CUSTOM_CZ_DATASET/Semantic segmentation dataset/Tile 8/images/VYSK111_0.jpg"
# image_path = r"C:\Users\jiric\Documents\VUT\BP\OBRAZKY\BIG_FORMAT_RESULTS\CUSTOM\WRTO24.2022.OLOM88.jpg"
# image_path = r"C:\Users\jiric\Documents\VUT\BP\OBRAZKY\BIG_FORMAT_RESULTS\CUSTOM\WRTO24.2022.VYSK11.jpg"


image_BGR = cv2.imread(image_path)
print("original image:", image_BGR.shape)
image_RGB = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)

height = image_RGB.shape[0]
width = image_RGB.shape[1]

new_h = height - (height % patch_size)
new_w = width - (width % patch_size)

image_RGB = (image_RGB[: new_h, :new_w, :])/255

image_RGB = image_RGB.astype(np.float32)

print("cropped image:", image_RGB.shape)

patches = patchify(image_RGB,(patch_size, patch_size, 3), step=patch_size)
print("initial patches:", patches.shape)

pred_image = np.zeros_like(image_RGB)

"""
plt.imshow(image_RGB)
plt.show()
"""



original image: (2560, 2560, 3)
cropped image: (2560, 2560, 3)
initial patches: (5, 5, 1, 512, 512, 3)


'\nplt.imshow(image_RGB)\nplt.show()\n'

In [31]:
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):

        print(i,j)

        patch = patches[i,j]

        predicted_patch = model.predict(patch)
        

        predicted_patch = predicted_patch[0,:,:,:]

        class_matrix = np.argmax(predicted_patch, axis=-1)

        predicted_patch_rgb = np.zeros((patch_size, patch_size,3))

        for class_id, color in colormap.items():
            predicted_patch_rgb[class_matrix==class_id]=color

        pred_image[i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size] = predicted_patch_rgb




print(pred_image.shape)


bgr_pred = cv2.cvtColor(pred_image, cv2.COLOR_RGB2BGR)

# cv2.imwrite("C:/Users/jiric/Documents/VUT/BP/DATASETS/CUSTOM_CZ_DATASET/Semantic segmentation dataset/pred_512_10_korDS_CE.png", bgr_pred)

0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4
(2560, 2560, 3)


In [32]:

# print(np.unique(image_RGB))
# pred_image = pred_image/255
# print(np.unique(pred_image))
# print(np.unique(bgr_pred))


In [33]:
"""
plt.figure(figsize=(15, 5))

plt.subplot(1,2,1)
plt.imshow(image_RGB)
plt.title('Cropped image')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(pred_image)
plt.title('Cropped prediction')
plt.axis('off')

plt.show()

# print(np.unique(image_RGB))
# print(np.unique(pred_image))

print(path_model)
"""

"\nplt.figure(figsize=(15, 5))\n\nplt.subplot(1,2,1)\nplt.imshow(image_RGB)\nplt.title('Cropped image')\nplt.axis('off')\n\nplt.subplot(1,2,2)\nplt.imshow(pred_image)\nplt.title('Cropped prediction')\nplt.axis('off')\n\nplt.show()\n\n# print(np.unique(image_RGB))\n# print(np.unique(pred_image))\n\nprint(path_model)\n"

In [34]:
if save_bool:
    cv2.imwrite(save_path, bgr_pred)
    print("IMAGE SAVED")
else:
    print("IMAGE NOT SAVED")

IMAGE SAVED
